In [ ]:
import cv2
from classify import cnn_model_fn,input_fn,label_dict,image_size
import tensorflow as tf
import numpy as np
from mtcnn.mtcnn import MTCNN

fer_pre_trained_model = '/Users/vimal/git/face_expression_recognisation/output/20190125-110715/'
face_expression_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,warm_start_from=fer_pre_trained_model)
detector = MTCNN()
video = cv2.VideoCapture('/Users/vimal/Documents/videos/hotel_mumbai.mp4')
while(video.isOpened()):
    # Capture frames
    _, frame = video.read()
    # Use MTCNN for detecting faces
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            #gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bounding_box = person['box']
            #keypoints = person['keypoints']
            
            x = bounding_box[0]
            y = bounding_box[1]
            w = bounding_box[0]+bounding_box[2]
            h = bounding_box[1]+bounding_box[3]
            
            face_image = frame[y:h, x:w]
            expression=''
            if face_image is not None and face_image.shape[0] > image_size and face_image.shape[1] > 64:
                #face_image = cv2.cvtColor(face_image,cv2.IMREAD_GRAYSCALE)
                face_image_save = cv2.resize(face_image,(image_size,image_size),interpolation = cv2.INTER_AREA)
                face_image = np.array(face_image_save,dtype=np.float32)
                predict_results = face_expression_classifier.predict(input_fn=lambda: input_fn(np.array([face_image])))
                for predict in predict_results:
                    expression = label_dict[np.argmax(predict['probabilities'])]
                    print(expression)

            # display resulting frame
            cv2.rectangle(frame,(x,y),(w,h),(0,155,155),2)
            cv2.putText(frame,expression,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,155,155))
            cv2.imshow('frame',frame)
            #file_name = "/Users/vimal/git/face_expression_recognisation/input/new/face_"+ str(y) +"_"+expression+"_.jpg"
            #cv2.imwrite(file_name, face_image_save)
            if cv2.waitKey(1) &0xFF == ord('q'):
                break
video.release()
cv2.destroyAllWindows()        

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/y3/d9nfl45x7p765ck8c1v_hlsc0000gp/T/tmpbgf7pikk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x145cc9908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:


INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_5/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_5/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_6/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_6/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_7/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_7/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-startin

INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_5/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_5/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_6/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting 

INFO:tensorflow:Warm-starting from: ('/Users/vimal/git/face_expression_recognisation/output/20190125-110715/',)
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_4/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_5/kernel; prev_var_name: Unchanged


INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/Users/vimal/git/face_expression_recognisation/output/20190125-110715/', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/Users/vimal/git/face_expression_recognisation/output/20190125-110715/',)
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_3/bias; prev_var_name: Un

neutral
neutral
neutral
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/y3/d9nfl45x7p765ck8c1v_hlsc0000gp/T/tmpbgf7pikk, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/Users/vimal/git/face_expression_recognisation/output/20190125-110715/', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/Users/vimal/git/face_expression_recognisation/output/20190125-110715/',)
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_v

INFO:tensorflow:Warm-starting variable: logits/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: logits/bias; prev_var_name: Unchanged
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
neutral
neutral
neutral
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/y3/d9nfl45x7p765ck8c1v_hlsc0000gp/T/tmpbgf7pikk, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/Users/vimal/git/face_expression_recognisation/output/20190125-110715/', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/Users/vimal/git/face_expression_recognisation/output/20190125-110715/',)
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-star

INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: logits/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: logits/bias; prev_var_name: Unchanged
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
neutral
neutral
neutral
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/y3/d9nfl45x7p765ck8c1v_hlsc0000gp/T/tmpbgf7pikk, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/Users/vimal/git/face_expression_recognisation/output/20190125-110715/', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_t

INFO:tensorflow:Warm-starting variable: conv2d_9/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_9/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: logits/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: logits/bias; prev_var_name: Unchanged
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
angry
neutral
neutral
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/y3/d9nfl45x7p765ck8c1v_hlsc0000gp/T/tmpbgf7pikk, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling mod